In [ ]:
import os
import numpy as np
from PIL import Image
import pandas as pd
import cv2

# Define directories
data_dir = './videos/MRI515_T2'
label_dir = os.path.join(data_dir, 'label_in_png_L4L5_renamed')

prompt_dir = 'manual_prompt_frame0_1_2'
prompt_dir = 'manual_prompt_frame0_1'
prompt_dir = 'manual_prompt_frame0'

# Directory containing auto-segmentation results (using the "nolap" version)
auto_seg_dir = os.path.join(data_dir, prompt_dir, 'SAM2_seg_mask_nolap')

# Output CSV file path (final long-format results)
csv_write_path = os.path.join(data_dir, prompt_dir, prompt_dir + '_area_results_long.csv')

# Check if required directories exist
if not os.path.exists(label_dir) or not os.path.exists(auto_seg_dir):
    raise FileNotFoundError("One or more required directories do not exist!")

# Each pixel corresponds to the actual area (e.g., unit in square millimeters)
spacing_factor = 0.6875 * 0.6875  # = 0.47265625

# Define target pixel values corresponding to 4 classes
classes = [50, 100, 150, 200]

results = []

# Process each label file
for label_file in os.listdir(label_dir):
    # Assume file name format is "123.png", extract numeric part as index
    try:
        label_index = int(os.path.splitext(label_file)[0])
    except ValueError:
        print(f"Skipping file with non-numeric name: {label_file}")
        continue

    # Construct full path for label file
    label_path = os.path.join(label_dir, label_file)
    label_index = str(int(label_index))
    
    # Construct full paths for auto-segmentation masks (4 classes)
    auto_seg_paths = [
        os.path.join(auto_seg_dir, f"frame_{label_index}_obj_{i}.png")
        for i in range(1, len(classes) + 1)
    ]
    
    # Check if all auto-segmentation files exist
    if not all(os.path.exists(p) for p in auto_seg_paths):
        print(f"Auto segmentation files missing for {label_file}, skipping.")
        continue

    # Read label mask
    label_mask = np.array(Image.open(label_path))

    # Read all auto-segmentation masks
    auto_masks = [np.array(Image.open(p)) for p in auto_seg_paths]

    file_result = {"file_name": label_file}

    # Compute area for each class
    for idx, class_val in enumerate(classes, start=1):
        # Label: Apply threshold, select pixels close to the target value (e.g., ±10)
        label_class_mask = (np.abs(label_mask - class_val) < 10).astype(np.uint8)
        
        # Auto segmentation: Corresponding obj_i file, using the same threshold condition
        auto_class_mask = (np.abs(auto_masks[idx - 1] - class_val) < 10).astype(np.uint8)
        
        # Resize auto-segmentation mask to match label mask
        target_size = (label_mask.shape[1], label_mask.shape[0])
        auto_class_mask = cv2.resize(auto_class_mask, target_size, interpolation=cv2.INTER_NEAREST)

        # Compute area: non-zero pixel count × spacing_factor
        label_area = np.sum(label_class_mask) * spacing_factor
        auto_area  = np.sum(auto_class_mask)  * spacing_factor

        # Store results with keys such as "class1_label_area", "class1_auto_area"
        file_result[f"class{idx}_label_area"] = label_area
        file_result[f"class{idx}_auto_area"] = auto_area

    results.append(file_result)
    
    # Print processing results
    print(f"Processed {label_file}:", end=" ")
    for idx in range(1, len(classes) + 1):
        print(f"Class{idx} -> Label: {file_result[f'class{idx}_label_area']:.2f}, Auto: {file_result[f'class{idx}_auto_area']:.2f}; ", end="")
    print()

# Convert wide-format results to long format
rows = []
for res in results:
    pid = os.path.splitext(res["file_name"])[0]  # Use file name (without extension) as PID
    for cls in range(1, len(classes) + 1):
        rows.append({
            "PID": pid,
            "Class": cls,
            "Auto Area": res[f"class{cls}_auto_area"],
            "Manual Area": res[f"class{cls}_label_area"]
        })

results_long_df = pd.DataFrame(rows)

# Save long-format results to CSV
results_long_df.to_csv(csv_write_path, index=False)
print(f"Results saved to {csv_write_path}")

# Output preview
print(results_long_df.head())


In [ ]:
results_long_df

In [ ]:
manual_prompt_id = ['00000', '00001', '00002']

# Logic: Determine which prefixes to remove from manual_prompt_id based on whether "1" or "2" is in prompt_dir
remove_prefixes = manual_prompt_id[: 1 + ("1" in prompt_dir) + ("2" in prompt_dir)]
print("Remove prefixes:", remove_prefixes)

# Filter function to remove rows where "PID" starts with any of the specified prefixes
filter_pid = lambda df: df[~df["PID"].astype(str).str.startswith(tuple(remove_prefixes))]

# Apply filtering
results_df = filter_pid(results_long_df)
results_df


In [ ]:
import numpy as np
import pandas as pd
import pingouin as pg
from scipy.stats import ttest_ind, ks_2samp, shapiro

# Assuming df_results already contains the segmentation area calculation results
# and includes columns like "PID", "Class", "Manual Area", "Auto Area", etc.

# ----------------- Statistical Tests, MAPE, and ICC -----------------
# Lists to store MAPE and ICC results for each class
mape_results = []  # Format: [Class, MAPE, MAPE_std]
icc_results = []   # Format: [Class, ICC, ICC_95%_CI_Lower, ICC_95%_CI_Upper]
# Dictionary to store p-values for each class
p_value_dict = {}

df_results = results_df

# Perform statistical tests, error analysis, and ICC calculations for each class
for class_value in [1, 2, 3, 4]:
    df_class = df_results[df_results["Class"] == class_value]
    if df_class.empty:
        continue

    # Extract manual segmentation (Manual Area) and automatic segmentation (Auto Area) values
    label_values = df_class["Manual Area"].values * 100
    auto_values = df_class["Auto Area"].values * 100

    # -------------- Normality Test --------------
    shapiro_label = shapiro(label_values)
    shapiro_auto_seg = shapiro(auto_values)

    label_is_normal = shapiro_label.pvalue >= 0.05
    auto_seg_is_normal = shapiro_auto_seg.pvalue >= 0.05

    print(f"\nClass {class_value}:")
    print(f"  Shapiro-Wilk Normality Test (Manual): p-value = {shapiro_label.pvalue:.3f} -> {'Normal' if label_is_normal else 'Non-Normal'}")
    print(f"  Shapiro-Wilk Normality Test (Auto-Seg): p-value = {shapiro_auto_seg.pvalue:.3f} -> {'Normal' if auto_seg_is_normal else 'Non-Normal'}")

    # -------------- Select Statistical Test Method --------------
    if label_is_normal and auto_seg_is_normal:
        t_stat, p_value = ttest_ind(label_values, auto_values, equal_var=False)
        print("  Method: Independent Two-Sample T-Test")
        print(f"  T-statistic: {t_stat:.4f}, P-value: {p_value:.3f}")
    else:
        ks_stat, p_value = ks_2samp(label_values, auto_values)
        print("  Method: Kolmogorov-Smirnov Test")
        print(f"  KS-statistic: {ks_stat:.4f}, P-value: {p_value:.3f}")

    # Store p-value for later summary
    p_value_dict[class_value] = p_value

    # -------------- Interpret Test Results --------------
    if p_value < 0.05:
        print("  - Significant difference detected (p < 0.05).")
    else:
        print("  - No significant difference detected (p >= 0.05).")

    # -------------- Compute MAPE --------------
    # Avoid division by zero using a small epsilon
    epsilon = 1e-6
    ape = np.abs((auto_values - label_values) / (label_values + epsilon))
    mape = np.mean(ape)
    mape_std = np.std(ape)
    mape_results.append([class_value, mape, mape_std])

    # -------------- Compute ICC --------------
    df_long = df_class.melt(id_vars=["PID"], value_vars=["Manual Area", "Auto Area"],
                            var_name="Method", value_name="Area")
    icc_df = pg.intraclass_corr(data=df_long, targets="PID", raters="Method", ratings="Area")
    # Extract the row corresponding to ICC2
    icc_row = icc_df[icc_df["Type"] == "ICC2"].iloc[0]
    icc_value = icc_row["ICC"]
    if "CI95%" in icc_row:
        ci_lower, ci_upper = icc_row["CI95%"]
    else:
        ci_lower, ci_upper = np.nan, np.nan
    icc_results.append([class_value, icc_value, ci_lower, ci_upper])

# ----------------- Display MAPE and ICC Results -----------------
mape_df = pd.DataFrame(mape_results, columns=['Class', 'MAPE', 'MAPE_std'])
icc_df = pd.DataFrame(icc_results, columns=['Class', 'ICC', 'ICC_95%_CI_Lower', 'ICC_95%_CI_Upper'])

# Convert MAPE and its standard deviation to percentage format (rounded to one decimal place)
mape_df['MAPE'] = (mape_df['MAPE'] * 100).round(1)
mape_df['MAPE_std'] = (mape_df['MAPE_std'] * 100).round(1)

mape_df = mape_df.sort_values(by="Class")
icc_df = icc_df.sort_values(by="Class")

# Print MAPE results in the format: 14.9 ± 13.1%
print("\n### MAPE Results ###")
for _, row in mape_df.iterrows():
    print(f"Class {int(row['Class'])}: {row['MAPE']} ± {row['MAPE_std']}%")

# Print ICC results (including 95% confidence interval)
print("\n### ICC Results ###")
for _, row in icc_df.iterrows():
    print(f"Class {int(row['Class'])}: {row['ICC']:.3f}, [{row['ICC_95%_CI_Lower']:.2f} - {row['ICC_95%_CI_Upper']:.2f}]")

# ----------------- Summary of p-values -----------------
# Define class names (adjust if necessary)
class_label_mapping = {
    1: "Right LES",
    2: "Right MF",
    3: "Left MF",
    4: "Left LES"
}
print("\n### p-values Summary ###")
# Print first row: Class names with corresponding numbers
header = " ".join([f"{class_label_mapping[cv]} {cv}" for cv in [1, 2, 3, 4]])
# Print second row: Corresponding p-values rounded to three decimals
p_values_line = " ".join([f"{p_value_dict.get(cv, np.nan):.3f}" for cv in [1, 2, 3, 4]])
print(header)
print(p_values_line)
